<a href="https://colab.research.google.com/github/EmperoR1127/ml_project/blob/emperor/h1b_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
#load the dataset
path_16 = "/content/drive/My Drive/Data/H-1B_Disclosure_RAW_Data_FY16.csv"
path_17 = "/content/drive/My Drive/Data/H-1B_Disclosure_RAW_Data_FY17.csv"
path_18 = "/content/drive/My Drive/Data/H-1B_Disclosure_RAW_Data_FY18.csv"
df_16 = pd.read_csv(path_16, encoding='utf-8')
df_17 = pd.read_csv(path_17, encoding='utf-8')
df_18 = pd.read_csv(path_18, encoding='utf-8')
#combine datesets and recalculate row index
df = pd.concat([df_16, df_17, df_18], sort=False, axis=0).reset_index(drop=True)
processed_data = df.copy()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#-----------PROCESS BINARY-VALUE COLUMNS-----------#
#fill missing values in columns of H-1B_DEPENDENT and FULL_TIME_POSITION with most frequent values
processed_data = processed_data.fillna(processed_data['H-1B_DEPENDENT'].value_counts().index[0])
processed_data = processed_data.fillna(processed_data['FULL_TIME_POSITION'].value_counts().index[0])
#numericalize colums of CASE_STATUS, FULL_TIME_POSITION and H-1B_DEPENDENT
numericalize_columns = {"CASE_STATUS":{"CERTIFIED":0, "DENIED":1}, 
                        "FULL_TIME_POSITION":{"Y":1, "N":0},
                        "H-1B_DEPENDENT":{"Y":1, "N":0}}
processed_data.replace(numericalize_columns, inplace=True)

In [0]:
#-----------PROCESS TIME-VALUE COLUMNS-----------#
#numericalize colums of DECISION_DATE, EMPLOYMENT_START_DATE and EMPLOYMENT_END_DATE
processed_data['DECISION_DATE'] = pd.to_datetime(processed_data['DECISION_DATE'],infer_datetime_format=True,errors='coerce').map(lambda x: str(x)[:4])
processed_data['EMPLOYMENT_START_DATE'] = pd.to_datetime(processed_data['EMPLOYMENT_START_DATE'],infer_datetime_format=True,errors='coerce').map(lambda x: str(x)[:4])
processed_data['EMPLOYMENT_END_DATE'] = pd.to_datetime(processed_data['EMPLOYMENT_END_DATE'],infer_datetime_format=True,errors='coerce').map(lambda x: str(x)[:4])
#drop NaT rows because we can't "guess" the specific date
processed_data = processed_data[processed_data.DECISION_DATE != 'NaT']
processed_data = processed_data[processed_data.EMPLOYMENT_START_DATE != 'NaT']
processed_data = processed_data[processed_data.EMPLOYMENT_END_DATE != 'NaT']
processed_data['DECISION_DATE'] = processed_data['DECISION_DATE'].astype(int)
processed_data['EMPLOYMENT_START_DATE'] = processed_data['EMPLOYMENT_START_DATE'].astype(int)
processed_data['EMPLOYMENT_END_DATE'] = processed_data['EMPLOYMENT_END_DATE'].astype(int)
#add one column as EMP_PERIOD, and drop EMPLOYMENT_START_DATE and EMPLOYMENT_END_DATE
processed_data['EMP_PERIOD'] = processed_data['EMPLOYMENT_END_DATE'] - processed_data['EMPLOYMENT_START_DATE']
processed_data = processed_data.drop(["EMPLOYMENT_START_DATE", "EMPLOYMENT_END_DATE"], axis=1)

In [0]:
#-----------PROCESS STRING-VALUE COLUMNS-----------#
#numericalize colums of EMPLOYER_NAME
processed_data['EMPLOYER_NAME'] = processed_data['EMPLOYER_NAME'].astype('category')
#employer_name_dict stores the mapping of numerical values to categorical values of EMPLOYER_NAME, incase of further reference
employer_name_dict = dict(zip(processed_data['EMPLOYER_NAME'].cat.codes, processed_data['EMPLOYER_NAME']))
processed_data['EMPLOYER_NAME'] = processed_data['EMPLOYER_NAME'].cat.codes
#numericalize colums of JOB_TITLE
processed_data['JOB_TITLE'] = processed_data['JOB_TITLE'].astype('category')
#employer_name_dict stores the mapping of numerical values to categorical values of JOB_TITLE, incase of further reference
job_title_dict = dict(zip(processed_data['JOB_TITLE'].cat.codes, processed_data['JOB_TITLE']))
processed_data['JOB_TITLE'] = processed_data['JOB_TITLE'].cat.codes
#concatenate the first 2 digit of column SOC_CODE
processed_data['SOC_CODE'] = processed_data['SOC_CODE'].map(lambda x: str(x)[:2])
#remove impurity in the column
#processed_data['SOC_CODE'] = pd.to_numeric(processed_data.SOC_CODE, errors='coerce')
processed_data = processed_data[processed_data.SOC_CODE != 'N']
processed_data = processed_data[processed_data.SOC_CODE != 'na']
processed_data = processed_data[processed_data.SOC_CODE != 'SO']
processed_data = processed_data[processed_data.SOC_CODE != 'CO']
processed_data = processed_data[processed_data.SOC_CODE != 'EL']
processed_data = processed_data[processed_data.SOC_CODE != '1-']
processed_data = processed_data[processed_data.SOC_CODE != 'EN']
processed_data = processed_data[processed_data.SOC_CODE != 'AI']
processed_data = processed_data[processed_data.SOC_CODE != 'ME']
processed_data = processed_data[processed_data.SOC_CODE != 'AC']
processed_data['SOC_CODE'] = processed_data['SOC_CODE'].astype('int')
#concatenate the first 2 digit of column NAIC_CODE
processed_data['NAIC_CODE'] = processed_data['NAIC_CODE'].map(lambda x: str(x)[:2])
#remove impurity in the column
processed_data = processed_data[processed_data.NAIC_CODE != 'N']
processed_data = processed_data[processed_data.NAIC_CODE != '&N']
#processed_data = processed_data[processed_data.NAIC_CODE != '0.']
processed_data['NAIC_CODE'] = processed_data['NAIC_CODE'].astype('int')
#numericalize colums of WORKSITE_STATE
processed_data['WORKSITE_STATE'] = processed_data['WORKSITE_STATE'].astype('category')
#worksite_state_dict stores the mapping of numerical values to categorical values of Worksite_STATE_full, incase of further reference
worksite_state_dict = dict(zip(processed_data['WORKSITE_STATE'].cat.codes, processed_data['WORKSITE_STATE']))
processed_data['WORKSITE_STATE'] = processed_data['WORKSITE_STATE'].cat.codes.astype('int')

In [0]:
#-----------PROCESS FLOAT-VALUE COLUMNS-----------#
#remove impurity in the column
processed_data = processed_data[processed_data.PW_UNIT_OF_PAY != 'N']
processed_data = processed_data[processed_data.PREVAILING_WAGE != 'N']
#according to google, there are 2080 working hours per year
pw_unit_column = {"PW_UNIT_OF_PAY":{"Year":1, "Hour":2080, "Month":12, "Week":52, "Bi-Weekly":26}}
processed_data.replace(pw_unit_column, inplace=True)
processed_data['PW_UNIT_OF_PAY'] = processed_data['PW_UNIT_OF_PAY'].astype('int')
#remove ',' in the column value
processed_data['PREVAILING_WAGE'] = processed_data['PREVAILING_WAGE'].astype('str')
processed_data['PREVAILING_WAGE'] = processed_data.PREVAILING_WAGE.str.replace(',','')
processed_data['PREVAILING_WAGE'] = processed_data['PREVAILING_WAGE'].astype('float')
#add one column as ANNUAL_SALARY
processed_data['ANNUAL_SALARY'] = processed_data['PREVAILING_WAGE'] * processed_data['PW_UNIT_OF_PAY']
processed_data = processed_data.drop(["PREVAILING_WAGE", "PW_UNIT_OF_PAY"], axis=1)

In [0]:
#-----------SET TRAIN_LABELS AND TRAIN_SET-----------#
train_labels = processed_data.copy()['CASE_STATUS']
train_labels = train_labels.astype('category')
train_set = processed_data.copy().drop(["CASE_STATUS"], axis=1)

In [8]:
#deal with the curse of imbalance combining over- and under-sampling using SMOTE and Edited Nearest Neighbours
from imblearn.combine import SMOTEENN
from collections import Counter
sme = SMOTEENN(random_state=42)
train_set_res, train_labels_res = sme.fit_resample(train_set, train_labels)
print(sorted(Counter(train_labels_res).items()))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


[(0, 1471563), (1, 1516668)]


In [12]:
train_set_res

array([[2.01500000e+03, 9.90870000e+04, 2.89780000e+04, ...,
        4.00000000e+00, 3.00000000e+00, 1.87200000e+05],
       [2.01500000e+03, 7.71590000e+04, 1.10901000e+05, ...,
        5.20000000e+01, 2.00000000e+00, 2.41842000e+05],
       [2.01500000e+03, 1.27160000e+05, 2.97370000e+04, ...,
        4.00000000e+00, 2.00000000e+00, 9.99860000e+04],
       ...,
       [2.01621270e+03, 6.55126834e+04, 8.88224315e+03, ...,
        3.15555873e+01, 1.57460499e+00, 4.96908418e+04],
       [2.01716911e+03, 6.01872391e+04, 1.70194166e+05, ...,
        4.93382127e+01, 2.83089363e+00, 6.01958480e+04],
       [2.01700000e+03, 8.98500000e+04, 1.18312000e+05, ...,
        9.00000000e+00, 3.00000000e+00, 6.65810000e+04]])

In [17]:
#univariate feature selection
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif
train_set_reduction = SelectPercentile(f_classif, percentile=90).fit_transform(train_set_res, train_labels_res)
train_set_reduction.shape

(2988231, 9)

In [18]:
#L1-based feature selection
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_set_res, train_labels_res)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(train_set_res)
X_new.shape

(2988231, 7)

In [19]:
#Tree-based feature selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(train_set_res, train_labels_res)
clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(train_set_res)
X_new.shape

(2988231, 4)